# 🎦**Web Scraping Movies Data From AlloCiné.fr**🎬

This script builds a DataFrame by web scraping the **movies** data from AlloCiné — a company which provides information on French cinema. Because of the long delay, we choose to scrape the data in two steps : 
- First, we scrape the url of each movie with `getMoviesUrl(...)`
- Lastly, we use the url list to scrape the data for each movie with `ScrapeURL(...)`

*📝Note 1: We use the popular BeautifulSoup package*

## **Functions :**

### `getMoviesUrl(start_page, end_page, nb_pages)` :

Returns the list of movies urls and a list of the movies that have not been released yet. It saves the list of movies urls in a CSV file as `../Movies/Data/movie_url.csv`. The argument must be integers and are used to select the range of pages (or the number of pages, default 1) you want to scrape the data from. The `end_page` is included. If the `end_page` value is correct (not `None` or >= `start_page`), the `nb_pages` argument is ignored. We only keep the urls of the movies which **have already been released**.

### `ScrapeURL(movie_url, dwld_poster)` :

Iterate over the list of urls generated by `getMoviesUrl(...)` and scrape the data for each movie. In the process, we extract :

- `id`: AlloCiné movie id
- `title`: the movie title (in French)
- `release_date`: the movie release date
- `duration`: the movie length (in minutes)
- `genres`: the movie genres (as a CSV string)
- `directors`: movie directors (as a CSV string)
- `actors`: main movie actors (as a CSV string)
- `nationality`: nationality of the movie (as a CSV string)
- `press_rating`: average press rating (from 0.5 to 5 stars ⭐⭐⭐⭐⭐)
- `nb_press_rating`: number of ratings made by the press
- `spect_rating`: average AlloCiné users rating (from 0.5 to 5 stars ⭐⭐⭐⭐⭐)
- `nb_spect_rating`: number of ratings made by the users/spectators
- `summary`: the movie summary (in French)
- `poster_link`: url of the movie poster

*📝Note 2: We can choose to download the poster image with the `dwld_poster` argument. If `True`, the poster image is downloaded and saved in the `../Movies/Posters/` folder.*

*📝Note 3: Intermediate functions were created to retrieve each individual feature of the dataframe, which makes it easier for debugging.*

The function `ScrapeURL(...)` returns two objects : the data as a dataframe and the url list of errors as a list. In addition the two objects are saved as `../Movies/Data/allocine_movies.csv` and `../Movies/Data/allocine_errors.csv`. Until now, the remaining errors were passed into ScrapeURL(...) for debugging and understanding what went wrong. The generated error messages were used to correct the functions in which the error occured. The objective is to retrieve the maximum number of movies in the first place, before proceeding to a data cleaning.

(⚠️ **Warning** ⚠️: the process can take a while, depending on the number of pages you choose to scrape. It is recommended to use a dedicated computer for this process, as it can take a long time to complete.)

---
## **Import libs**

In [1]:
# Import libs
import pandas as pd
import numpy as np
import re
import unicodedata
from tqdm.notebook import tqdm_notebook as tqdm
from time import time, sleep
from datetime import timedelta, datetime, date
from urllib.request import urlopen
from random import randint
from bs4 import BeautifulSoup
import dateparser
import os

from warnings import warn, filterwarnings
from IPython.display import clear_output
import traceback

## **Functions:** Getting movie infos

### Get movie ID: `get_movie_ID(movie_url)`

In [2]:
def get_movie_ID(movie_url: str) -> int:
    '''
    Get the movie ID from the movie page url.
    :param movie_url: URL of the movie page.
    :return: The movie ID.'''
    movie_ID = int(re.sub(r"\D", "", movie_url))
    return movie_ID

### Get movie title: `get_movie_title(movie_soup)`

In [3]:
def get_movie_title(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie title from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie title.'''
    movie_title = movie_soup.find("div", {"class": "titlebar-title"}).text.strip()
    return movie_title

### Get movie release date: `get_movie_release_date(movie_soup)`

#### Convert French months to English months: `convert_month(month)`

In [4]:
def convert_month(month: str) -> str:
    '''
    Convert French months into English months for the dateparser to work.
    :param month: French month.
    :return: English month.'''
    if month == "janvier":
        return "January"
    elif month == "février":
        return "February"
    elif month == "mars":
        return "March"
    elif month == "avril":
        return "April"
    elif month == "mai":
        return "May"
    elif month == "juin":
        return "June"
    elif month == "juillet":
        return "July"
    elif month == "août":
        return "August"
    elif month == "septembre":
        return "September"
    elif month == "octobre":
        return "October"
    elif month == "novembre":
        return "November"
    elif month == "décembre":
        return "December"
    else:
        return "Unknown"

#### Main function: `get_movie_release_date(movie_soup)`

In [5]:
def get_movie_release_date(movie_soup: BeautifulSoup) -> date:
    '''
    Scrape the movie release date from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie release date.'''
    movie_release_date = movie_soup.find("span", {"class": "date"})
    if movie_release_date:
        try:
            movie_release_date = movie_release_date.text.strip()
            month = movie_release_date.split(' ')[1]
            movie_release_date = movie_release_date.replace(month, convert_month(month))
            movie_release_date = dateparser.parse(movie_release_date, date_formats=["%d %B %Y"]).date()      
        except:
            warn(f'The date is not in the expected format! The movie is skipped.')
            return None  # If the date is not in the expected format, return None
    return movie_release_date

### Get movie duration: `get_movie_duration(movie_soup)`

In [6]:
def get_movie_duration(movie_soup: BeautifulSoup) -> int:
    '''
    Scrape the movie duration from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie duration in minutes if exists. Else None.'''
    movie_duration_str = movie_soup.find("span", {"class": "spacer"}).next_sibling.strip()
    movie_duration = None
    if movie_duration_str != "":
        duration_timedelta = pd.to_timedelta(movie_duration_str).components
        movie_duration = int(duration_timedelta.hours * 60 + duration_timedelta.minutes)
    return movie_duration

### Get movie genres: `get_movie_genres(movie_soup)`

In [7]:
def get_movie_genres(movie_soup: BeautifulSoup) -> str:
    ''' 
    Scrape the movie genres from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie genres as a CVS string. None if no genres are found.'''
    div_genres = movie_soup.find("div", {"class": "meta-body-item meta-body-info"})
    if div_genres:
        movie_genres = [
            genre.text
            for genre in div_genres.find_all("span", class_=re.compile(r".*==$"))
            if "\n" not in genre.text
        ]
        return ", ".join(movie_genres)
    return None

### Get movie directors: `get_movie_directors(movie_soup)`

In [8]:
def get_movie_directors(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie directors from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie directors as a CSV string. None if no directors are found.'''
    div_directors = movie_soup.find_all(
            "div", {"class": "meta-body-item meta-body-direction"}
        )
    # We retrieve the people next to the "Par" and "De" keywords, 
    # and we keep only one instance of each person.
    if div_directors:
        movie_directors = [
            link.text
            for directors in div_directors
            for link in directors.find_all(
                ["a", "span"], class_=re.compile(r".*blue-link$")
            )
        ]
        return ", ".join(set(movie_directors))
    return None

### Get movie actors: `get_movie_actors(movie_soup)`

In [9]:
def get_movie_actors(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie actors from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie actors as a CSV string. None if no actors are found.'''
    div_actors = movie_soup.find("div", {"class": "meta-body-item meta-body-actor"})
    if div_actors:
        # We retrieve the people next to the "Avec" keyword
        movie_actors = [actor.text for actor in div_actors.find_all(["a", "span"])][1:]
        return ", ".join(movie_actors)
    return None

### Get movie nationality: `get_movie_nationality(movie_soup)`

In [10]:
def get_movie_nationality(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie nationality from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie nationality as a CSV string.'''
    movie_nationality = [
            nationality.text.strip()
            for nationality in movie_soup.find_all("span", class_="nationality")
        ]
    return ", ".join(movie_nationality)

### Get movie press ratings: `get_movie_press_rating(movie_soup)`

In [11]:
def get_movie_press_rating(movie_soup: BeautifulSoup) -> float:
    '''
    Scrape the movie average press rating from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie average press rating. None if no rating is found.'''
    movie_ratings = movie_soup.find_all("div", class_="rating-item")
    for ratings in movie_ratings:
        if "Presse" in ratings.text:
            # eg: Change from 4,5 to 4.5.
            return float(
                re.sub(
                    ",", ".", ratings.find("span", {"class": "stareval-note"}).text
                )
            )
    return None

### Get movie number of press ratings: `get_movie_press_rating_count(movie_soup)`

In [12]:
def get_movie_press_rating_count(movie_soup: BeautifulSoup) -> int:
    '''
    Scrape the movie number of press ratings from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie number of press ratings. None if no rating is found.'''
    movie_ratings = movie_soup.find_all("div", class_="rating-item")
    for ratings in movie_ratings:
        if "Presse" in ratings.text:
            # We keep only the number of ratings, and we leave the number of reviews out.
            # (eg: re.match("\s\d+", " 10154 notes dont 1327 critiques").group() returns " 10154")
            return int(
                re.match(
                    r"\s\d+",
                    ratings.find("span", {"class": "stareval-review"}).text,
                ).group()
            )
    return None

### Get movie spectator ratings: `get_movie_spect_rating(movie_soup)`

In [13]:
def get_movie_spect_rating(movie_soup: BeautifulSoup) -> float:
    '''
    Scrape the movie average spectators' rating from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie average spec rating. None if no rating is found.'''
    # get all the available ratings
    movie_ratings = movie_soup.find_all("div", class_="rating-item")
    for ratings in movie_ratings:
        if "Spectateurs" in ratings.text:
            # eg: Change from 4,5 to 4.5.
            return float(
                re.sub(
                    ",", ".", ratings.find("span", {"class": "stareval-note"}).text
                )
            )
    return None

### Get movie number of spec ratings: `get_movie_spect_rating_count(movie_soup)`

In [14]:
def get_movie_spect_rating_count(movie_soup: BeautifulSoup) -> int:
    '''
    Scrape the movie number of spec ratings from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie number of spec ratings. None if no rating is found.'''
    movie_ratings = movie_soup.find_all("div", class_="rating-item")
    for ratings in movie_ratings:
        if "Spectateurs" in ratings.text:
            # We keep only the number of ratings, and we leave the number of reviews out.
            # (eg: re.match("\s\d+", " 10154 notes dont 1327 critiques").group() returns " 10154")
            return int(
                re.match(
                    r"\s\d+",
                    ratings.find("span", {"class": "stareval-review"}).text
                ).group()  
            )
    return None

### Get movie summary: `get_movie_summary(movie_soup)`

In [15]:
def get_movie_summary(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie summary from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie summary. None if no summary is found.'''
    movie_summary = movie_soup.find("section", {"class": "section ovw ovw-synopsis"})
    # Check if the synopsis division exists 
    if movie_summary:
        movie_summary = movie_summary.find("div", {"class": "content-txt"})
        # Check if the synopsis text exists
        if movie_summary:
            movie_summary = movie_summary.text.strip()
            return unicodedata.normalize("NFKC", movie_summary)
    return None

### Get movie poster: `get_movie_poster(movie_soup)`

In [16]:
def get_movie_poster(movie_soup: BeautifulSoup) -> str:
    '''
    Scrape the movie poster link from the movie page.
    :param movie_soup: BeautifulSoup object of the movie page.
    :return: The movie poster link.'''
    return movie_soup.find("img", {"class": "thumbnail-img"})["src"]

### Download movie poster: `download_movie_poster(poster_link, movie_title, movie_id)`

In [30]:
def download_movie_poster(poster_link: str, movie_title: str, movie_id: int=None) -> None:
    '''
    Download the movie poster from the poster link.
    :param poster_link: The poster link.
    :param movie_title: The movie title.
    :param movie_id: The movie id.
    :return: nothing but saves the movie poster as a jpg file.'''
    poster = get_url(poster_link)
    poster_path = "../Movies/Posters/"
    # We create the folder if not exists
    os.makedirs(os.path.dirname(poster_path), exist_ok=True) 
    movie_title = re.sub(r"( |:|,|'|\?|\\|/|\*|<|>|\")", "_", movie_title)
    save_path = f"{poster_path}{movie_title}_poster_{movie_id}.jpg"
    with open(save_path, "wb") as f:
        f.write(poster.read())
    f.close()

#### Download all movie posters: `download_all_posters(movies_df)`

In [51]:
def download_all_posters(movies_df: pd.DataFrame) -> None:
    '''
    Download all the movie posters from the poster links in the movies dataframe.
    :param movies_df: The movies dataframe.
    :return: nothing but saves the movie posters as jpg files.'''
    for index, row in movies_df.iterrows():
        download_movie_poster(row["poster_link"], row["title"], row["id"])
        sleep(0.1)

---
## **Function:** `getMoviesUrl(start_page, end_page, nb_pages)`

### Get url: `get_url(url)`

In [33]:
def get_url(url: str):
    '''
    Try to get and open the page url.
    :param url: url to get the content from.
    :return: the content of the url page
    '''
    page = ''
    while page == '':
        try:
            page = urlopen(url)
            break
        except:
            print(f"Error occured when opening the url.\nURL: {url}")
            print("\nConnection refused by the server...")
            print("Let's wait for 5 seconds")
            print("ZZzzzz...")
            sleep(5)
            print("Ok, now let's try again...\n")
            continue
    return page

### Main function: `getMoviesUrl(start_page, end_page, nb_pages)`

In [20]:
def getMoviesUrl(start_page: int, end_page: int=None, nb_pages: int=1) -> None:
    '''
    Scrape the movies urls from the AlloCine website's movie page (http://www.allocine.fr/films/).
    The range of pages to scrape goes from start_page to end_page (included) if end_page is not None or <= start_page.
    Else, the range of pages to scrape goes from start_page to start_page + nb_pages.
    It will ignore the movies that has not been released yet.
    :param start_page: The first page to scrape.
    :param end_page: The last page to scrape (included) (optional).
    :param nb_pages: The number of pages to scrape (default 1).
    :return: List of movies_url and a list of the movies that have not been released yet.
    Saves the list of movies urls in a csv file.'''
    if start_page <= 0:
        return ValueError('start_page must be positive !')       

    # We ignore dateparse warnings
    filterwarnings("ignore",message="The localize method is no longer necessary, as this time zone supports the fold attribute")
    
    # Set the list
    movie_url = []

    # Preparing the setting and monitoring of the loop
    start_time = time()
    p_requests = start_page
    # We will scrape by default at least 1 page if end_page is not specified, 
    # or if it is lower than start_page,
    # or if the number of pages to scrape is negative.
    if nb_pages < 1:
        nb_pages = 1
    if end_page == None or end_page < start_page:
        end_page = start_page + nb_pages - 1
    
    # Number of movie requests
    m_requests = 0
    # Store the series not yet released 
    nr_movies = []
    
    # Loop over the pages
    # Monitoring the loop with tqdm
    for p in tqdm(range(start_page, end_page + 1), desc="Fetching movies urls", unit="pages"):

        # Get request
        url = f'http://www.allocine.fr/films/?page={p}'
        response = get_url(url)
        
        # Pause the loop
        sleep(randint(1,2))
            
        # Monitoring the requests
        elapsed_time = time() - start_time
        #print(f'>Page Request: {p_requests}; Frequency: {p_requests/elapsed_time} requests/s')
            
        # Warning for non-200 status codes
        if response.status != 200:
            warn(f'>Page Request: {p_requests}; Status code: {response.status_code}')

        # Break the loop if the number of requests is greater than expected
        if p_requests > end_page:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        html_text = response.read().decode("utf-8")
        html_soup = BeautifulSoup(html_text, 'html.parser')

        # Select all the movies url from a single page
        movies = html_soup.find_all('h2', 'meta-title')
        m_requests += len(movies)
               
        # Monitoring the requests
        #print(f'>Page Request: {p_requests}; Movie Request: {m_requests}')
        
        # Pause the loop
        sleep(1)
        
        for movie in movies:
            m_url = f'http://www.allocine.fr{movie.a["href"]}'
            m_soup = BeautifulSoup(get_url(m_url), 'html.parser')
            m_rd = get_movie_release_date(m_soup)
            # We keep the movie url only if the movie has already been released.
            if m_rd != None and m_rd <= datetime.now().date():
                movie_url.append(m_url) 
            else:
                nr_movies.append(m_url)
        p_requests += 1
        print(f"{len(movie_url)}/{m_requests} URLs collected.")
        sleep(.1)

    # Saving the files
    movie_path = '../Movies/Data/'
    # We create the folder if not exists
    os.makedirs(os.path.dirname(movie_path), exist_ok=True) 
    print(f'--> Done; {p_requests-start_page} Page Requests and {m_requests-len(nr_movies)}/{m_requests} Movie Requests in {timedelta(seconds=time()-start_time)}\n--> {len(nr_movies)} movies not yet released.')
    r = np.asarray(movie_url)
    np.savetxt(f"{movie_path}movie_url.csv", r, delimiter=",", fmt='%s')
    return movie_url, nr_movies

## **Function:** `ScrapeURL(movie_url, dwld_poster)`

In [21]:
def ScrapeURL(movie_url: list, dwld_poster: bool = False) -> tuple[pd.DataFrame, pd.DataFrame]:
    '''
    Scrape the data from the movies page urls.
    :param movie_url: The list of movies page urls.
    :param dwld_poster: Boolean to download the movie poster (default False).
    :return: The dataframe of the movies and the dataframe of the urls that return an error. 
    Both are saved into csv files.'''
    # init the dataframe
    c = ["id",
        "title",
        "release_date",
        "duration",
        "genres",
        "directors",
        "actors",
        "nationality",
        "press_rating",
        "nb_press_rating",
        "spect_rating",
        "nb_spect_rating",
        "summary",
        "poster_link"
    ]
    df = pd.DataFrame(columns=c)
    
    # preparing the setting and monitoring loop
    start_time = time()
    n_request = 0
    
    # init list to save errors
    errors = []
    
    # request loop
    for url in tqdm(movie_url, desc = "Scraping movies data"):
        try :
            response = get_url(url)

            # Pause the loop
            sleep(randint(1,2))

            # Monitoring the requests
            n_request += 1
            print
            
            elapsed_time = time() - start_time
            #print(f'Request #{n_request}; Frequency: {n_request/elapsed_time} requests/s')
            #clear_output(wait = True)

            # Pause the loop
            sleep(randint(1,2))

            # Warning for non-200 status codes
            if response.status != 200:
                warn(f'Request #{n_request}; Status code: {response.status_code}')
                errors.append(url)

            # Parse the content of the request with BeautifulSoup
            html_text = response.read().decode("utf-8")
            movie_html_soup = BeautifulSoup(html_text, 'html.parser')
            
            # Get the movie data
            if movie_html_soup.find('div', 'titlebar-title'):
                # Get the movie ID 
                tp_id = get_movie_ID(url)
                # Scrape the title
                tp_title = get_movie_title(movie_html_soup)
                # Scrape the release date
                tp_release_dt = get_movie_release_date(movie_html_soup)
                # Scrape the duration
                tp_duration = get_movie_duration(movie_html_soup)
                # Scrape the genres
                tp_genre = get_movie_genres(movie_html_soup)
                # Scrape the directors
                tp_director = get_movie_directors(movie_html_soup)
                # Scrape the actors
                tp_actor = get_movie_actors(movie_html_soup)
                # Scrape the nationality
                tp_nation = get_movie_nationality(movie_html_soup)
                # Scrape the press ratings
                tp_press_rating = get_movie_press_rating(movie_html_soup)
                # Scrape the number of press ratings
                tp_nb_press_rating = get_movie_press_rating_count(movie_html_soup)
                # Scrape the spec ratings
                tp_spec_rating = get_movie_spect_rating(movie_html_soup)
                # Scrape the number of spec ratings
                tp_nb_spec_rating = get_movie_spect_rating_count(movie_html_soup)
                # Scrape the summary
                tp_summary = get_movie_summary(movie_html_soup)
                # Scrape the poster
                tp_poster = get_movie_poster(movie_html_soup)
                # Download the poster (optional)
                if dwld_poster:
                    download_movie_poster(tp_poster, tp_title)
                
                # Append the data
                df_tmp = pd.DataFrame({'id': [tp_id],
                                       'title': [tp_title],
                                       'release_date': [tp_release_dt],
                                       'duration': [tp_duration],
                                       'genres': [tp_genre],
                                       'directors': [tp_director],
                                       'actors': [tp_actor],
                                       'nationality': [tp_nation],
                                       'press_rating': [tp_press_rating],
                                       'nb_press_rating': [tp_nb_press_rating],
                                       'spect_rating': [tp_spec_rating],
                                       'nb_spect_rating': [tp_nb_spec_rating],
                                       'summary': [tp_summary],
                                       'poster_link': [tp_poster]})
                
                df = pd.concat([df, df_tmp], ignore_index=True)                
        except:
            errors.append(url)
            warn(f'Request #{n_request} fail; Total errors : {len(errors)}')
            # traceback.print_exc()
            
    # monitoring 
    movie_path = '../Movies/Data/'
    # We create the folder if not exists
    os.makedirs(os.path.dirname(movie_path), exist_ok=True) 
    elapsed_time = time() - start_time
    print(f'--> Done; {n_request} requests in {timedelta(seconds=elapsed_time)} with {len(errors)} errors\n--> {len(df)} movies successfully scraped')
    # Saving files
    df.to_csv(f"{movie_path}allocine_movies.csv", index=False)
    # list to dataframe
    errors_df = pd.DataFrame(errors, columns=['url'])
    errors_df.to_csv(f"{movie_path}allocine_errors.csv",index=False,header=False)
    print(f"Files saved as '{movie_path}allocine_movies.csv' and '{movie_path}allocine_errors.csv'")
    # return dataframe and errors
    return df, errors

---
## **Launching the script**

### Getting movies urls

In [22]:
# Scrape the page from start_page to end_page (included) or with nb_pages
start_page = 1
end_page = None
nb_pages = 5
movie_url, nr_movies = getMoviesUrl(start_page, end_page=end_page, nb_pages=nb_pages)

Fetching movies urls:   0%|          | 0/5 [00:00<?, ?pages/s]

15/15 URLs collected.
28/30 URLs collected.
41/45 URLs collected.
55/60 URLs collected.
69/75 URLs collected.
--> Done; 5 Page Requests and 69/75 Movie Requests in 0:01:37.143771
--> 6 movies not yet released.


### Loading the list of urls

In [52]:
# Load the list of urls 
m_url = pd.read_csv("../Movies/Data/movie_url.csv",names=['url'])
m_url = m_url['url'].tolist()

### Scraping the data

In [24]:
# Scrape the data 
movies_df, movies_errors = ScrapeURL(m_url)

Scraping movies data:   0%|          | 0/69 [00:00<?, ?it/s]

--> Done; 69 requests in 0:04:17.865416 with 0 errors
--> 69 movies successfully scraped
Files saved as '../Movies/Data/allocine_movies.csv' and '../Movies/Data/allocine_errors.csv'
